In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
pd.set_option('display.max_colwidth', None)

# First approach - CountVectorizer

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
print(train_df.shape)
print(test_df.shape)

(7613, 5)
(3263, 4)


In [3]:
#What CountVectorizer does is to create a "vocabulary list" with all the words used in all the given data (all the words of all sencente: twitts) in this case it makes 
# a total of 21637 words. Afterwards the function compares this vocabulary list with every sentence and set 1 when the word of the sentence appears in the vocabulary
# list (it can be 2 if the word appear twice and so on). This means we will have for every sentence a lenth of 21637.

In [4]:
count_vectorizer = feature_extraction.text.CountVectorizer()

## let's get counts for the first 5 tweets in the data
example_train_vectors = count_vectorizer.fit_transform(train_df["text"][0:5])

In [5]:
#we use .todense() here because these vectors are "sparse" (only non-zero elements are kept to save space)
print('this is the first sentence:  ',train_df["text"][0])
print('\nthese are the dimensions of the vector of the first sentence: ',example_train_vectors[0].todense().shape)
print('\nthis is the vector for the first sentence: \n',example_train_vectors[0].todense())
print('\nthis is the vocabulary list for the 5 first sentences (same dimensions that vector): \n',count_vectorizer.vocabulary_)

this is the first sentence:   Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all

these are the dimensions of the vector of the first sentence:  (1, 54)

this is the vector for the first sentence: 
 [[0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
  0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0]]

this is the vocabulary list for the 5 first sentences (same dimensions that vector): 
 {'our': 34, 'deeds': 12, 'are': 5, 'the': 49, 'reason': 39, 'of': 29, 'this': 50, 'earthquake': 13, 'may': 25, 'allah': 4, 'forgive': 18, 'us': 52, 'all': 3, 'forest': 17, 'fire': 16, 'near': 26, 'la': 24, 'ronge': 42, 'sask': 44, 'canada': 11, 'residents': 41, 'asked': 7, 'to': 51, 'shelter': 47, 'in': 21, 'place': 37, 'being': 8, 'notified': 28, 'by': 9, 'officers': 30, 'no': 27, 'other': 33, 'evacuation': 14, 'or': 31, 'orders': 32, 'expected': 15, '13': 1, '000': 0, 'people': 35, 'receive': 40, 'wildfires': 53, 'california': 10, 'just': 23, 'got': 20, 'sent': 46, 'photo

In [6]:
train_vectors = count_vectorizer.fit_transform(train_df["text"])

## note that we're NOT using .fit_transform() here. Using just .transform() makes sure
# that the tokens in the train vectors are the only ones mapped to the test vectors - 
# i.e. that the train and test vectors use the same set of tokens.
test_vectors = count_vectorizer.transform(test_df["text"])

In [7]:
print(train_vectors.todense().shape)
print(test_vectors.todense().shape)

(7613, 21637)
(3263, 21637)


In [8]:
print(train_vectors[0].todense().shape)
print(train_vectors[0].todense())
#print('\nthis is the vocabulary list all sentences (same dimensions that vector): \n',count_vectorizer.vocabulary_)

(1, 21637)
[[0 0 0 ... 0 0 0]]


In [9]:
count_vectorizer.transform(test_df["text"]).todense().shape

(3263, 21637)

In [10]:
## Our vectors are really big, so we want to push our model's weights
## toward 0 without completely discounting different words - ridge regression 
## is a good way to do this.
clf = linear_model.RidgeClassifier()

In [11]:
print(train_vectors.shape)
print(train_df["target"].shape)

(7613, 21637)
(7613,)


In [12]:
scores = model_selection.cross_val_score(clf, train_vectors, train_df["target"], cv=3, scoring="f1")
scores

array([0.59453669, 0.56455572, 0.64082434])

In [13]:
clf.fit(train_vectors, train_df["target"])

RidgeClassifier()

In [14]:
sample_submission = pd.read_csv("sample_submission.csv")


In [15]:
sample_submission["target"]

0       0
1       1
2       1
3       0
4       1
       ..
3258    1
3259    1
3260    1
3261    1
3262    0
Name: target, Length: 3263, dtype: int64

In [16]:
sample_submission.to_csv("sample_submission_1.csv",index=False)

In [17]:
sample_submission

,id,target
0,0,0
1,2,1
2,3,1
3,9,0
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [18]:
test = pd.read_csv("sample_submission_1.csv")

# Second approach - TF-IDF

In [19]:
# What TF-IDF does is to create a "vocabulary vector" with all the words used in all the given data (all the words of all sencente: twitts) in this case it makes 
# a total of 21637 words. Afterwards the function compares this "vocabulary vector" with every sentence and set a number of importance (IDF value)
# to every word of the "vocabulary vector" in the respect of the analysed sentence

In [27]:

from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer


In [21]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
print(train_df.shape)
print(test_df.shape)

(7613, 5)
(3263, 4)


In [22]:
train_vectors = count_vectorizer.fit_transform(train_df["text"])

## note that we're NOT using .fit_transform() here. Using just .transform() makes sure
# that the tokens in the train vectors are the only ones mapped to the test vectors - 
# i.e. that the train and test vectors use the same set of tokens.
test_vectors = count_vectorizer.transform(test_df["text"])

print(train_vectors[0].todense().shape)
print(train_vectors[0].todense())
#print('\nthis is the vocabulary list all sentences (same dimensions that vector): \n',count_vectorizer.vocabulary_)

(1, 21637)
[[0 0 0 ... 0 0 0]]


In [23]:
# Let's do an example with the 5 first tweets to see how it works
train_vectors=TfidfVectorizer()
result=train_vectors.fit_transform(train_df["text"][0:5])
print(train_df["text"][0])
print(result[0].todense().shape)
print(result[0].todense())
print(train_vectors.vocabulary_)


Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all
(1, 54)
[[0.         0.         0.         0.23336118 0.28924517 0.23336118
  0.         0.         0.         0.         0.         0.
  0.28924517 0.28924517 0.         0.         0.         0.
  0.28924517 0.         0.         0.         0.         0.
  0.         0.28924517 0.         0.         0.         0.28924517
  0.         0.         0.         0.         0.28924517 0.
  0.         0.         0.         0.28924517 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.28924517 0.23336118 0.         0.28924517 0.        ]]
{'our': 34, 'deeds': 12, 'are': 5, 'the': 49, 'reason': 39, 'of': 29, 'this': 50, 'earthquake': 13, 'may': 25, 'allah': 4, 'forgive': 18, 'us': 52, 'all': 3, 'forest': 17, 'fire': 16, 'near': 26, 'la': 24, 'ronge': 42, 'sask': 44, 'canada': 11, 'residents': 41, 'asked': 7, 'to': 51, 'shelter': 47, 'in': 21, 'place': 37, 'being': 8, 'notified': 28, 'by': 

In [28]:
# Now lets do it with all.
train_vectors=TfidfTransformer().fit_transform(train_df["text"])
test_vectors=TfidfTransformer().transform(test_df["text"])


ValueError: could not convert string to float: 'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [25]:
print(train_vectors.todense().shape)
print(test_vectors.todense().shape)

(7613, 21637)
(3263, 21637)


##### Our model

In [123]:
clf = linear_model.RidgeClassifier()

In [124]:
train_df["target"].shape

(7613,)

In [125]:
train_vectors.shape

(7613, 21637)

In [126]:
scores = model_selection.cross_val_score(clf, train_vectors, train_df["target"], cv=3, scoring="f1")
scores

array([0.63366337, 0.6122449 , 0.68442211])

In [70]:
clf.fit(train_vectors, train_df["target"])

RidgeClassifier()

In [71]:
sample_submission = pd.read_csv("sample_submission.csv")

In [72]:
clf.predict(test_vectors)

ValueError: X has 12279 features per sample; expecting 21637

In [ ]:
sample_submission.to_csv("sample_submission_TFIDF.csv",index=False)

In [ ]:
test = pd.read_csv("sample_submission_1.csv")

# N approach (using Spacy)